# Spark LLM Assistant

## Initialization

In [1]:
from langchain import OpenAI
from spark_llm import SparkLLMAssistant

llm = OpenAI(model_name='gpt-4') # using gpt-4 can achieve better results
assistant=SparkLLMAssistant(llm=llm, verbose=True, enable_cache=True)
assistant.activate() # active partial functions for Spark DataFrame

/opt/homebrew/lib/python3.8/site-packages/langchain/llms/openai.py:179: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/homebrew/lib/python3.8/site-packages/langchain/llms/openai.py:751: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/14 11:17:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Example 1: Auto sales by brand in US 2022

In [3]:
llm("who are u")

'I am an AI trained by OpenAI, here to assist and provide information to help with your questions and tasks.'

In [5]:
llm("who are u")


'I am an AI trained by OpenAI, here to assist and provide information to help with your questions and tasks.'

In [4]:
assistant.commit()

In [7]:
# Search and ingest web content into a DataFrame
auto_df = assistant.create_df("2022 USA national auto sales by brand")
auto_df.show()

InvalidRequestError: Unrecognized request argument supplied: cache

In [4]:
assistant._llm.predict("tell me a joke")

"Why don't scientists trust atoms? Because they make up everything!"

In [5]:
assistant._llm.predict("tell me a joke")


"Why don't scientists trust atoms? Because they make up everything!"

In [ ]:
auto_df.llm_plot()

In [ ]:
# Apply transforms to a Dataframe
auto_top_growth_df=auto_df.llm_transform("top brand with the highest growth")
auto_top_growth_df.show()

In [ ]:
# Explain what a DataFrame is retrieving.
auto_top_growth_df.llm_explain()

## Example 2: USA Presidents

In [ ]:
# You can also specify the expected columns for the ingestion.
df=assistant.create_df("USA presidents", ["president", "vice_president"])
df.show()

In [ ]:
presidents_who_were_vp = df.llm_transform("presidents who were also vice presidents")
presidents_who_were_vp.show()

In [ ]:
presidents_who_were_vp.llm_explain()

# Example 3: Top 10 tech companies

In [ ]:
# Search and ingest web content into a DataFrame
company_df=assistant.create_df("Top 10 tech companies by market cap", ['company', 'cap', 'country'])
company_df.show()

In [ ]:
us_company_df=company_df.llm_transform("companies in USA")
us_company_df.show()

In [ ]:
us_company_df.llm_explain()

In [ ]:
us_company_df.llm_plot()

## Example 4: Ingestion from a URL
Instead of searching for the web page, you can also ask the assistant to ingest from a URL.

In [ ]:
assistant.create_df('https://time.com/6235186/best-albums-2022/').show()